In [5]:
!git clone https://github.com/facebookresearch/segment-anything-2.git

fatal: destination path 'segment-anything-2' already exists and is not an empty directory.


In [8]:
import cv2
import torch
import numpy as np
from segment_anything import SamPredictor, sam_model_registry

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_checkpoint = "sam_vit_b_01ec64.pth"  # Provide the path to the small SAM model checkpoint
model_type = "vit_b"  # Use "vit_b" for the small version

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

# Open webcam
cap = cv2.VideoCapture(0)

In [13]:
def detect_color(mask, image):
    """Detect the average color of the masked region"""
    # Create a mask of the region
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    # Convert to HSV color space for better color detection
    hsv = cv2.cvtColor(masked_image, cv2.COLOR_BGR2HSV)

    # Calculate the average color in the masked area
    mean_color = cv2.mean(hsv, mask=mask)
    return mean_color

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for faster processing
    resized_frame = cv2.resize(frame, (256, 256))
    resized_frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    # Use SAM to predict the mask for the frame
    predictor.set_image(resized_frame_rgb)
    masks, scores, logits = predictor.predict(point_coords=None, point_labels=None, multimask_output=True)

    # Loop over the masks (choose the highest score mask for simplicity)
    best_mask_index = np.argmax(scores)
    best_mask = masks[best_mask_index].astype(np.uint8) * 255

    # Detect the color of the segmented object
    mean_color = detect_color(best_mask, resized_frame)

    # Display the results
    cv2.imshow("Original Frame", frame)
    cv2.imshow("Segmented Mask", best_mask)

    # Display color in HSV format
    print(f"Detected HSV Color: {mean_color}")

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Detected HSV Color: (77.27839667977283, 57.01277850589777, 64.93042813455658, 0.0)
Detected HSV Color: (42.93906588308073, 8.456387256418187, 169.53263223012684, 0.0)
Detected HSV Color: (39.94195568400771, 8.56791907514451, 170.743978805395, 0.0)
Detected HSV Color: (66.58704008786381, 6.285557386051621, 172.2720117151748, 0.0)
Detected HSV Color: (0.0, 0.0, 0.0, 0.0)
Detected HSV Color: (0.0, 0.0, 0.0, 0.0)
